In [1]:
from collections import OrderedDict
import os

from matplotlib import pyplot as plt

os.environ['CUDA_VISIBLE_DEVICES'] = '3'
import os
import yaml
import pandas as pd
import torch
import shap
from evaluate import get_trained_model
from utils import *
from dateset import load_full_features

### 生成平均重要分数

In [2]:
for disease in ["EW-T2D", "LC", "C-T2D", "IBD", "Obesity"]:
    dfs = []
    for seed in [392, 412, 432, 452, 472]:
        path = f"/hdd/wmh/MSFT-Transfromer/explain/{disease}/MTMFTransformer/{seed}.csv"
        dfs.append(pd.read_csv(path))
    numpy_arrays = [df.to_numpy() for df in dfs]
    sum_array = np.sum(numpy_arrays, axis=0)

    # 检查
    for sid in range(dfs[0].shape[0]):
        for featid in range(dfs[0].shape[1]):
            sum = 0
            for i in range(5):
                sum += numpy_arrays[i][sid][featid]
            if sum != sum_array[sid][featid]:
                assert 0
                
    df = pd.DataFrame(sum_array)
    path = f"/hdd/wmh/MSFT-Transfromer/explain/{disease}/MTMFTransformer/overall.csv"
    df.to_csv(path, index=False)

### 根据重要听得分绘制图像

In [ ]:
for disease in ["EW-T2D", "LC", "C-T2D", "IBD", "Obesity"]:
    ko_feat = list(pd.read_csv(f"/hdd/wmh/MSFT-Transfromer/Data/{disease}/ko_abundance.csv").columns)[2:]
    species_feat = list(pd.read_csv(f"/hdd/wmh/MSFT-Transfromer/Data/{disease}/species_abundance.csv").columns)[2:]
    feat_names = ko_feat + species_feat
    # 去除过长featname
    feat_names = [x if len(x) < 20 else x[x.rfind('|') + 1:] for x in feat_names]

    # 读取重要性得分
    shap_values = pd.read_csv(f"/hdd/wmh/MSFT-Transfromer/explain/{disease}/MTMFTransformer/overall.csv").values

    # 绘制图像
    shap.summary_plot(shap_values, x_test, feature_names=feat_names, rng=np.random.default_rng(), show=False)
    plt.savefig(f"/hdd/wmh/MSFT-Transfromer/explain/{disease}/MTMFTransformer/overall.png", dpi=720)

### 提取重要性得分比较高的特征

In [38]:
for disease in ["EW-T2D", "LC", "C-T2D", "IBD", "Obesity"]:
    shap_values = pd.read_csv(f"/hdd/wmh/MSFT-Transfromer/explain/{disease}/MTMFTransformer/overall.csv").values
    shap_values = np.sum(np.abs(shap_values), axis=0)
    ko_feat = list(pd.read_csv(f"/hdd/wmh/MSFT-Transfromer/Data/{disease}/ko_abundance.csv").columns)[2:]
    species_feat = list(pd.read_csv(f"/hdd/wmh/MSFT-Transfromer/Data/{disease}/species_abundance.csv").columns)[2:]
    feat_names = ko_feat + species_feat
    # 去除过长featname
    # feat_names = [x if len(x) < 20 else x[x.rfind('|') + 1:] for x in feat_names]

    faet_importance = {k: v for (k, v) in zip(feat_names, shap_values)}

    # 使用 sorted 根据字典的值进行降序排序
    sorted_dict = dict(sorted(faet_importance.items(), key=lambda item: item[1], reverse=True))

    df = pd.DataFrame(list(sorted_dict.items()), columns=['feat', 'importance'])
    df.to_csv(f"/hdd/wmh/MSFT-Transfromer/explain/{disease}/MTMFTransformer/feat_importance.csv", index=False)

In [26]:
shap_values.shape

(1584,)